In [1]:
import cv2

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
from mtcnn.mtcnn import MTCNN

Using TensorFlow backend.
/home/skb/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/skb/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/skb/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/skb/.local/lib/python3.6/site-packages/tensorflow/python/framework/d

In [5]:
detector=MTCNN()

In [57]:
#sun=cv2.imread('../Pictures/ocv/sunglasses.png', cv2.IMREAD_UNCHANGED)
sun=cv2.imread('../Pictures/ocv/blueglasses.png', cv2.IMREAD_UNCHANGED)

sun2=sun
overlay_mask=sun2[:,:,3:]
overlay_img=sun2[:,:,:3]
background_mask = 255 - overlay_mask
overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)
overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

In [79]:
#video=cv2.VideoCapture(0)
video=cv2.VideoCapture('../Pictures/ocv/manface.mp4')

c = 0
while True:
    ret, frame = video.read()
    if not ret: break
        
      
    result=detector.detect_faces(frame)
    
    
    faces_no=len(result)
    facebox=[]
    lefteye=[]
    righteye=[]
    nose=[]
    mouthleft=[]
    mouthright=[]
    
    for i in range(0,faces_no):
        facebox.append(result[i]['box'])
        lefteye.append(result[i]['keypoints']['left_eye'])
        righteye.append(result[i]['keypoints']['right_eye'])
        nose.append(result[i]['keypoints']['nose'])
        mouthleft.append(result[i]['keypoints']['mouth_left'])
        mouthright.append(result[i]['keypoints']['mouth_right'])

        #facebox.append(result[i]['box'])
    
    new_frame=(frame/255).astype(np.float32)
    
    for i in facebox:
        cv2.rectangle(frame,(i[0],i[1]),((i[0]+i[2]),(i[1]+i[3])),(255, 0, 0),5)
        sun_width=int(i[2])
        sun_height=int(i[3]/3)
        
    for i in lefteye:
        cv2.circle(frame,i,5,(255, 0, 0),5)
        
    for i in righteye:
        cv2.circle(frame,i,5,(255, 0, 0),5)
        
    if(len(righteye)!=0):
        sun2=cv2.resize(sun, (sun_width, sun_height)) 
        overlay_mask=sun2[:,:,3:]
        overlay_img=sun2[:,:,:3]
        background_mask = 255 - overlay_mask
        overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
        background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)
        overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))
    
        height,width,channels=sun2.shape
        roi=frame[int(righteye[0][1]-(sun_height/3)):int(height+righteye[0][1]-(sun_height/3)),int(lefteye[0][0]-(sun_width/4)):int(width+lefteye[0][0]-(sun_width/4))]
        #roi=frame[0:height,0:width]
        face_part = (roi * (1 / 255.0)) * (background_mask * (1 / 255.0))
        dst=cv2.add(face_part,overlay_part)
        new_frame[int(righteye[0][1]-(sun_height/3)):int(height+righteye[0][1]-(sun_height/3)),int(lefteye[0][0]-(sun_width/4)):int(width+lefteye[0][0]-(sun_width/4))]=dst
        #new_frame[0:height,0:width]=dst
    
    cv2.namedWindow('frame',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('frame',600,600)
    cv2.imshow('frame', new_frame)
    key = cv2.waitKey(3)
    if key == 27: break
        
cv2.destroyAllWindows()
video.release()

In [74]:
video.release()


In [75]:
cv2.destroyAllWindows()



In [76]:
nose

[(447, 132)]